# Data Preprocessing: Data Cleaning and Feature Generation 

In [1]:
import numpy as np
import pandas as pd
import os, datetime
import warnings
warnings.filterwarnings('ignore')


## ===== Hyper-parameters =====

In [2]:
# ======= price_features
bid_price_features = ['Bid1', 'Bid2', 'Bid3', 'Bid4', 'Bid5'] 
ask_price_features = ['Ask1', 'Ask2', 'Ask3', 'Ask4', 'Ask5']


# ======= size_features
bid_size_features = ['Bid1Size', 'Bid2Size', 'Bid3Size', 'Bid4Size', 'Bid5Size']
ask_size_features = ['Ask1Size', 'Ask2Size', 'Ask3Size', 'Ask4Size', 'Ask5Size']

                      
# ======= Data Path
processed_data_path = 'features/'
if not os.path.isdir(processed_data_path):
    os.makedirs(processed_data_path)
    print("made folder:", processed_data_path)

## ===== Load data =====

In [3]:
df_book = pd.read_parquet('./book')
df_trade = pd.read_parquet('./trade')

In [4]:
df_book = df_book.sort_values(by=['TimeStamp'])
df_book

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,...,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt
index,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,...,1111.21,3000,1117.99,8358,1111.11,1600,1118.00,1300,1114.015,1115.265
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,...,1113.00,1900,1117.40,4000,1112.22,2800,1117.50,100,1115.265,1116.495
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,...,1113.01,600,1117.40,4000,1113.00,2000,1117.50,100,1116.495,1113.795
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,...,1112.22,2800,1117.40,4000,1112.00,1600,1117.50,100,1113.795,1111.565
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,...,1111.21,3000,1117.00,300,1111.18,200,1117.40,4000,1111.565,1114.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,1588229807743561,1265.68,300,1265.69,100,1265.59,200,1265.70,3200,1265.46,...,1265.03,100,1265.88,1400,1265.02,200,1265.90,2100,1265.685,1265.690
4698,1588229810757082,1265.68,200,1265.70,3200,1265.59,200,1265.87,100,1265.46,...,1265.02,200,1265.90,2100,1265.01,400,1266.00,1000,1265.690,1265.690
4699,1588229813401635,1265.68,200,1265.70,3100,1265.60,100,1265.87,100,1265.59,...,1265.46,100,1265.90,2100,1265.02,200,1266.00,1000,1265.690,1265.785


In [ ]:
df_book.describe()

### Generate next_Bid1 and next_Ask1 column 

In [ ]:
# generate next_Bid1 next_Ask1 column 
tmp_list = df_book['Bid1'][1:].values.tolist()
tmp_list.append(np.nan)
df_book = df_book.assign(nextBid1=tmp_list)

# generate next_Ask1 column 
tmp_list = df_book['Ask1'][1:].values.tolist()
tmp_list.append(np.nan)
df_book = df_book.assign(nextAsk1=tmp_list)

In [ ]:
# df_trade

## ===== Clean Data =====

### Book data

In [ ]:
pd.set_option('use_inf_as_na', True)

In [ ]:
# ----- check null, na, nan 

print('df_book.isnull().sum()', df_book.isnull().sum())
print('df_book.isnan().sum()', df_book.isna().sum())
print("df_book.isna().sum()", df_book.isna().sum())

In [ ]:
# Drop all rows with NaN values
# # Note: the nextAsk1 and nextBid1 in the next day will be droped here too
df_book.dropna(how='any', axis=0, inplace=True) 

In [ ]:
# check null, na, nan 

print('df_book.isnull().sum()', df_book.isnull().sum())
print('df_book.isnan().sum()', df_book.isna().sum())
print("df_book.isna().sum()", df_book.isna().sum())

In [ ]:
### Trade data

In [ ]:
# print('df_trade.isnull().sum()', df_trade.isnull().sum())
# print('df_trade.isnan().sum()', df_trade.isna().sum())
# print("df_trade.isna().sum()", df_trade.isna().sum())

In [ ]:
# # Drop all rows with NaN values
# df_trade.dropna(how='any', axis=0, inplace=True) 

In [ ]:
# # check null, na, nan after MA

# print('df_trade.isnull().sum()', df_trade.isnull().sum())
# print('df_trade.isnan().sum()', df_trade.isna().sum())
# print("df_trade.isna().sum()", df_trade.isna().sum())

## ====== Generate Features ======

### ----- Price Statistical Features ----- 

In [ ]:
df_book['Ask_Total_Size'] = df_book[ask_size_features].sum(axis=1)
df_book['Bid_Total_Size'] = df_book[bid_size_features].sum(axis=1)

In [ ]:
# ----- weighted avg. (mean) of Bid and Ask

df_book['Bid_Mean'] = (df_book['Bid1']*df_book['Bid1Size'] + df_book['Bid2']*df_book['Bid2Size'] + \
                             df_book['Bid3']*df_book['Bid3Size'] + df_book['Bid4']*df_book['Bid4Size'] + \
                             df_book['Bid5']*df_book['Bid5Size']) / df_book['Bid_Total_Size']


df_book['Ask_Mean'] = (df_book['Ask1']*df_book['Ask1Size'] + df_book['Ask2']*df_book['Ask2Size'] + \
                             df_book['Ask3']*df_book['Ask3Size'] + df_book['Ask4']*df_book['Ask4Size'] + \
                             df_book['Ask5']*df_book['Ask5Size']) / df_book['Ask_Total_Size']

df_book['Ask_Mean']

In [ ]:
# ----- microprice

# print(df_book)
df_book['MicroPrice'] = df_book['Ask1']*df_book['Ask1Size'] + df_book['Bid1']*df_book['Bid1Size']
df_book['MicroPrice'] = df_book['MicroPrice']/(df_book['Ask1Size'] + df_book['Bid1Size'])
# print(df_book)

### ----- Price Distance Features -----

In [ ]:
# ----- speard features

df_book['Spread1'] = (df_book['Ask1'] - df_book['Bid1']) / df_book['midpt']
df_book['Spread2'] = (df_book['Ask2'] - df_book['Bid2']) / df_book['midpt']
df_book['Spread3'] = (df_book['Ask3'] - df_book['Bid3']) / df_book['midpt']
df_book['Spread4'] = (df_book['Ask4'] - df_book['Bid4']) / df_book['midpt']
df_book['Spread5'] = (df_book['Ask5'] - df_book['Bid5']) / df_book['midpt']
df_book['SpreadMean'] = (df_book['Ask_Mean'] - df_book['Bid_Mean']) / df_book['midpt']

### ----- Size Ratio Features -----

In [ ]:
# ----- size ratio features

df_book['BidAskRatio1'] = df_book['Bid1Size'] / df_book['Ask1Size']
df_book['BidAskRatio2'] = df_book['Bid2Size'] / df_book['Ask2Size']
df_book['BidAskRatio3'] = df_book['Bid3Size'] / df_book['Ask3Size']
df_book['BidAskRatio4'] = df_book['Bid4Size'] / df_book['Ask4Size']
df_book['BidAskRatio5'] = df_book['Bid5Size'] / df_book['Ask5Size']
df_book['BidAskRatioTotal'] = df_book['Bid_Total_Size'] / df_book['Ask_Total_Size']

In [ ]:
# ----- Queue Imblance features

df_book['Q_ImB1'] = (df_book['Bid1Size'] - df_book['Ask1Size']) / (df_book['Bid1Size'] + df_book['Ask1Size'])
df_book['Q_ImB2'] = (df_book['Bid2Size'] - df_book['Ask2Size']) / (df_book['Bid2Size'] + df_book['Ask2Size'])
df_book['Q_ImB3'] = (df_book['Bid3Size'] - df_book['Ask3Size']) / (df_book['Bid3Size'] + df_book['Ask3Size'])
df_book['Q_ImB4'] = (df_book['Bid4Size'] - df_book['Ask4Size']) / (df_book['Bid4Size'] + df_book['Ask4Size'])
df_book['Q_ImB5'] = (df_book['Bid5Size'] - df_book['Ask5Size']) / (df_book['Bid5Size'] + df_book['Ask5Size'])

### ----- Size Distribution Features -----

In [ ]:
# Ask/Bid size distribution at time t 

print(df_book)  
for feat in ask_size_features:
    df_book[feat + 'Prop'] = df_book[feat]/df_book['Ask_Total_Size']
    
for feat in bid_size_features:
    df_book[feat + 'Prop'] = df_book[feat]/df_book['Bid_Total_Size']

In [ ]:
df_book

## Save processed_data

In [ ]:
df_book.to_csv(processed_data_path+'book.csv', index=False)
df_trade.to_csv(processed_data_path+'trade.csv', index=False)

## ======= Data pre-processed finished =======

In [74]:
df_book = pd.read_parquet('./book')
df_trade = pd.read_parquet('./trade')
df_trade

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice
index,,,,,,,
213,1585704600549504,99924,141932,0,0.00,100,1115.5
214,1585704600551309,142138,141795,100,1116.00,0,0.0
215,1585704601244086,86301,142421,0,0.00,100,1115.0
216,1585704601252795,141888,143205,0,0.00,100,1115.0
217,1585704601256878,141888,143923,0,0.00,200,1115.0
...,...,...,...,...,...,...,...
14431,1588229814066074,4769476,4762592,100,1265.70,0,0.0
14432,1588229814315798,4769476,4769545,0,0.00,100,1265.7
14433,1588229816071596,4769476,4770153,0,0.00,100,1265.7


In [75]:
df_book

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,...,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt
index,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,...,1111.21,3000,1117.99,8358,1111.11,1600,1118.00,1300,1114.015,1115.265
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,...,1113.00,1900,1117.40,4000,1112.22,2800,1117.50,100,1115.265,1116.495
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,...,1113.01,600,1117.40,4000,1113.00,2000,1117.50,100,1116.495,1113.795
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,...,1112.22,2800,1117.40,4000,1112.00,1600,1117.50,100,1113.795,1111.565
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,...,1111.21,3000,1117.00,300,1111.18,200,1117.40,4000,1111.565,1114.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,1588229807743561,1265.68,300,1265.69,100,1265.59,200,1265.70,3200,1265.46,...,1265.03,100,1265.88,1400,1265.02,200,1265.90,2100,1265.685,1265.690
4698,1588229810757082,1265.68,200,1265.70,3200,1265.59,200,1265.87,100,1265.46,...,1265.02,200,1265.90,2100,1265.01,400,1266.00,1000,1265.690,1265.690
4699,1588229813401635,1265.68,200,1265.70,3100,1265.60,100,1265.87,100,1265.59,...,1265.46,100,1265.90,2100,1265.02,200,1266.00,1000,1265.690,1265.785


In [76]:
df_trade['TimeStamp_s'] = df_trade['TimeStamp']//1000000
df_tmp = df_trade[['TimeStamp_s', 'BuyVolume', 'BuyPrice', 'SellVolume', 'SellPrice']]
df_book['TimeStamp_s'] = df_book['TimeStamp']//1000000

In [77]:
df_tmp.describe()

,TimeStamp_s,BuyVolume,BuyPrice,SellVolume,SellPrice
count,3.675590e+05,367559.000000,367559.000000,367559.000000,367559.000000
mean,1.587005e+09,87.848245,646.122228,74.669237,555.349449
std,7.724819e+05,231.774045,599.587418,218.285571,600.662141
min,1.585705e+09,0.000000,0.000000,0.000000,0.000000
25%,1.586326e+09,0.000000,0.000000,0.000000,0.000000
50%,1.587017e+09,100.000000,1128.000000,0.000000,0.000000
75%,1.587607e+09,100.000000,1199.890000,100.000000,1193.650000
max,1.588230e+09,20000.000000,1299.940000,16952.000000,1299.900000


In [78]:
df_tmp = df_tmp.groupby('TimeStamp_s').agg({'BuyVolume':'sum', 'BuyPrice':'max', 'SellVolume':'sum', 'SellPrice':'max'})
df_tmp

,BuyVolume,BuyPrice,SellVolume,SellPrice
TimeStamp_s,,,,
1585704600,100,1116.00,100,1115.50
1585704601,0,0.00,400,1115.00
1585704602,500,1115.99,2600,1115.00
1585704604,1800,1116.85,0,0.00
1585704605,1000,1116.85,100,1113.11
...,...,...,...,...
1588229813,100,1265.70,0,0.00
1588229814,3100,1265.70,100,1265.70
1588229816,0,0.00,100,1265.70


In [79]:
df_join = df_book.merge(df_tmp, how='left', on='TimeStamp_s')

In [80]:
df_join

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,...,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,TimeStamp_s,BuyVolume,BuyPrice,SellVolume,SellPrice
0,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,...,1600,1118.00,1300,1114.015,1115.265,1585704603,NaN,NaN,NaN,NaN
1,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,...,2800,1117.50,100,1115.265,1116.495,1585704606,300.0,1115.50,0.0,0.0
2,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,...,2000,1117.50,100,1116.495,1113.795,1585704609,1000.0,1117.00,100.0,1115.0
3,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,...,1600,1117.50,100,1113.795,1111.565,1585704612,NaN,NaN,NaN,NaN
4,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,...,200,1117.40,4000,1111.565,1114.200,1585704615,500.0,1116.95,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97413,1588229807743561,1265.68,300,1265.69,100,1265.59,200,1265.70,3200,1265.46,...,200,1265.90,2100,1265.685,1265.690,1588229807,NaN,NaN,NaN,NaN
97414,1588229810757082,1265.68,200,1265.70,3200,1265.59,200,1265.87,100,1265.46,...,400,1266.00,1000,1265.690,1265.690,1588229810,NaN,NaN,NaN,NaN
97415,1588229813401635,1265.68,200,1265.70,3100,1265.60,100,1265.87,100,1265.59,...,200,1266.00,1000,1265.690,1265.785,1588229813,100.0,1265.70,0.0,0.0
97416,1588229816412576,1265.70,1500,1265.87,100,1265.69,600,1265.88,200,1265.68,...,100,1266.13,100,1265.785,1265.790,1588229816,0.0,0.00,100.0,1265.7


In [81]:
print('df_join.isnull().sum()', df_join.isnull().sum())
print('df_join.isnan().sum()', df_join.isna().sum())
print("df_join.isna().sum()", df_join.isna().sum())

df_join.isnull().sum() TimeStamp          0
Bid1               0
Bid1Size           0
Ask1               0
Ask1Size           0
Bid2               0
Bid2Size           0
Ask2               0
Ask2Size           0
Bid3               0
Bid3Size           0
Ask3               0
Ask3Size           0
Bid4               0
Bid4Size           0
Ask4               0
Ask4Size           0
Bid5               0
Bid5Size           0
Ask5               0
Ask5Size           0
midpt              0
nextMidpt         21
TimeStamp_s        0
BuyVolume      46965
BuyPrice       46965
SellVolume     46965
SellPrice      46965
dtype: int64
df_join.isnan().sum() TimeStamp          0
Bid1               0
Bid1Size           0
Ask1               0
Ask1Size           0
Bid2               0
Bid2Size           0
Ask2               0
Ask2Size           0
Bid3               0
Bid3Size           0
Ask3               0
Ask3Size           0
Bid4               0
Bid4Size           0
Ask4               0
Ask4Size          